In [1]:
import os
import json
import gc
from unidecode import unidecode

In [2]:
wikidata_graph = {}
wikidata_name_id = {}
wikidata_en_graph = {}
wikidata_en_graph = {}
wikidata_text = {}
wikidata_en_text = {}
codex_graph = {}

In [3]:
#codex
# golden = {}
# entities = json.load(open('entities.json'))
# relations = json.load(open('relations.json'))
# types = json.load(open('types.json'))
# for key, value in entities.items():
#     golden[key] = value['label']
# for key, value in relations.items():
#     golden[key] = value['label']
# for key, value in types.items():
#     golden[key] = value['label']

In [4]:
find = ['P31', 'P279', 'P361'] # instance of, subclass of, part of, these relations are very vague so we discard them
with open('wikidata5m_all_triplet.txt', 'r') as file:
    for line in file:
        line = line.strip()
        line = line.split('\t')
        line = [x.strip() for x in line]
        if line[1] in find:
            continue
        if line[0] not in wikidata_graph:
            wikidata_graph[line[0]] = {}
        if line[1] not in wikidata_graph[line[0]]:
            wikidata_graph[line[0]][line[1]] = []
        wikidata_graph[line[0]][line[1]].append(line[2])

In [5]:
wikidata_graph['Q41828']

{'P19': ['Q193722'], 'P106': ['Q39631'], 'P27': ['Q159']}

In [5]:
#codex
# for suf in ['l','m','s']:
#     for file in os.listdir('/home/vvjain3/rag-llm-verify/codex-master/data/triples/codex-'+suf):
#         with open(os.path.join('/home/vvjain3/rag-llm-verify/codex-master/data/triples/codex-'+suf, file), 'r') as file:
#             for line in file:
#                 line = line.strip()
#                 line = line.split('\t')
#                 line = [x.strip() for x in line]
#                 if line[0] not in codex_graph:
#                     codex_graph[line[0]] = {}
#                 if line[1] not in codex_graph[line[0]]:
#                     codex_graph[line[0]][line[1]] = []
#                 codex_graph[line[0]][line[1]].append(line[2])

In [6]:
#only choosing relations are unique per entity
# double_val = []
# for key, value in wikidata_graph.items():
#     new_value = {}
#     for k, v in value.items():
#         if len(v) > 1:
#             double_val.append((key, k, v))
#             continue
#         new_value[k] = v[0].strip()
#     wikidata_graph[key] = new_value
# gc.collect()

0

In [7]:
with open('wikidata5m_text.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split('\t')
        line = [x.strip() for x in line]
        text = line[1].strip()
        if len(line) > 2:
            for i in range(2, len(line)):
                text += ' ' + line[i].strip()
        if line[0] in wikidata_graph:
            wikidata_text[line[0]] = text
gc.collect()

0

In [8]:
name_not_in_text = []
possible_entity_names = {}
with open('wikidata5m_entity.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split('\t')
        line = [x.strip() for x in line]
        if line[0] not in wikidata_text:
            continue
        possible_entity_names[line[0]] = line[1:]
        possible = line[1:min(30, len(line))] #select most common label from the first 30 aliases
        common = ''
        all_text = wikidata_text[line[0]]
        all_text = unidecode(all_text)
        all_text = all_text.lower()
        all_text = all_text.replace('–', '-') #replace en dash with hyphen
        min_ind = 1000000
        #choose common according to the order of the aliases appearing in the text
        for name in possible:
            name = name.lower()
            name = unidecode(name)
            if name in all_text:
                ind = all_text.index(name)
                if ind < min_ind:
                    min_ind = ind
                    common = name
                if min_ind <= 15:
                    break
        if common == '':
            name_not_in_text.append((line[0], all_text, line[1:], possible))
            continue
        wikidata_name_id[line[0]] = common.strip()
with open('wikidata5m_relation.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split('\t')
        line = [x.strip() for x in line]
        min_ind = 1000000
        wikidata_name_id[line[0]] = line[1]
gc.collect()

0

In [9]:
name_not_in_text[:5], len(name_not_in_text)

([('Q54319',
   'star wars (sutauozu, suta wozu) is a family computer video game released in 1987 by namco. despite being based on the first star wars film, some levels are based on the later two star wars films. it is the only game in the star wars franchise that was released exclusively in japan. the game is a common side-scrolling platformer where the player controls luke skywalker (appearing with black hair due to color limitations), as he travels to join the rebellion against the empire.',
   ['Star Wars (1987 video game)',
    'Star Wars (Namco video game)',
    'スターウォーズ',
    'star wars (1987 video game)',
    'star wars (namco video game)'],
   ['Star Wars (1987 video game)',
    'Star Wars (Namco video game)',
    'star wars (1987 video game)',
    'star wars (namco video game)']),
  ('Q24391822',
   'on 7 june 2016, at around 08:40 (utc+3), a bombing occurred in central istanbul, turkey, killing 12 people and injuring 51 others, three of them seriously. the attack targeted a 

In [10]:
# for key, value in golden.items():
#     wikidata_name_id[key] = value

In [11]:
len(wikidata_graph), len(codex_graph)

(4276852, 69684)

In [12]:
with open('wikidata5m_en_unidecoded.json', 'w') as f:
    json.dump(wikidata_graph, f)
with open('wikidata_name_id_uni.json', 'w') as f:
    json.dump(wikidata_name_id, f)
with open('wikidata_text_uni.json', 'w') as f:
    json.dump(wikidata_text, f)

In [13]:
temp = {}
for key, value in wikidata_graph.items():
    if len(value) > 0:
        temp[key] = value
len(temp)

4248110

In [14]:
len(wikidata_name_id)

3156188

In [15]:
wikidata_en_graph = {}

In [16]:
shortened = 0
total = 0
for key, value in wikidata_graph.items():
    if key not in wikidata_text or key not in wikidata_name_id:
        shortened += len(list(value.keys()))
        total += len(list(value.keys()))
        continue
    all_text = wikidata_text[key]
    all_text = unidecode(all_text)
    all_text = all_text.lower()
    all_text = all_text.replace('–', '-')
    new_value = {}
    for k, v in value.items():
        if k in wikidata_name_id and v in wikidata_name_id:
            if v in possible_entity_names:
                all_possible_vals = possible_entity_names[v]
            else:
                all_possible_vals = [wikidata_name_id[v]]
            for val in all_possible_vals:
                val = unidecode(val)
                val = val.lower()
                ind = all_text.find(val)
                if ind != -1:
                    new_value[wikidata_name_id[k]] = wikidata_name_id[v]
                    break
            # new_value[wikidata_name_id[k]] = wikidata_name_id[v]
            total += 1
    if len(new_value) > 0:
        wikidata_en_graph[wikidata_name_id[key]] = new_value
gc.collect()

0

In [17]:
len(wikidata_en_graph), shortened, total

(2783620, 3226599, 10751937)

In [18]:
with open('wikidata5m_en_unidecoded.json', 'w') as f:
    json.dump(wikidata_en_graph, f)

In [19]:
wikidata_id_name = {}
for key, value in wikidata_name_id.items():
    wikidata_id_name[value] = key

In [20]:
wikidata_graph_trimmed = {}
for key, value in wikidata_graph.items():
    if key in wikidata_name_id:
        if wikidata_name_id[key] in wikidata_en_graph:
            for k, v in value.items():
                if k in wikidata_name_id and v in wikidata_name_id:
                    if key not in wikidata_graph_trimmed:
                        wikidata_graph_trimmed[key] = {}
                    wikidata_graph_trimmed[key][k] = v

In [21]:
wikidata_en_graph_util = {}
for key, value in wikidata_en_graph.items():
    wikidata_en_graph_util[key] = {}
    for k, v in value.items():
        wikidata_en_graph_util[key][v] = k

In [22]:
wikidata5m_en_text = {}
for key, value in wikidata_text.items():
    if key in wikidata_name_id:
        wikidata5m_en_text[wikidata_name_id[key]] = value
len(wikidata5m_en_text), len(wikidata_text)

(3132737, 4191605)

In [23]:
with open('wikidata5m_en_text.json', 'w') as f:
    json.dump(wikidata5m_en_text, f)

In [24]:
with open('wikidata5m_en_util_unidecoded.json', 'w') as f:
    json.dump(wikidata_en_graph_util, f)

In [25]:
import os
import json
from utils import *

2024-01-31 23:54:04.263917: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 23:54:04.289948: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 23:54:04.420838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 23:54:04.420871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 23:54:04.438290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [26]:
#graph = json.load(open('wikidata5m_en_graph_util_trimmed.json'))
graph_algos = GraphAlgos(wikidata_en_graph_util)

In [29]:
query = graph_algos.generate_random_query(k=4, return_path=True)
query

(('What is the writing system of the official language of the country of dhubab?',
  4),
 'dhubab',
 'arabic alphabet',
 ['dhubab', 'Yemen', 'Arabic', 'arabic alphabet'])

In [5]:
wikidata_en_graph

{'lalit kumar goel': {'instance of': 'Q5'},
 'ugo riccarelli': {'languages spoken, written or signed': 'Q652',
  'place of death': 'Q220',
  'instance of': 'Q5',
  'place of birth': 'Q9284',
  'award received': 'Q731542',
  'country of citizenship': 'Q38',
  'occupation': 'Q36180'},
 'Road to Paradise (film)': {'director': 'Q1365729',
  'color': 'Q838368',
  'screenwriter': 'Q676186',
  'production designer': 'Q1700525',
  'original language of film or TV show': 'Q1860',
  'country of origin': 'Q30',
  'genre': 'Q130232',
  'director of photography': 'Q724695',
  'composer': 'Q1818548',
  'instance of': 'Q11424',
  'producer': 'Q1188369',
  'production company': 'Q126399',
  'movement': 'Q3408764'},
 'robert van der horst': {'country of citizenship': 'Q29999',
  'given name': 'Q4927937',
  'place of birth': 'Q9832',
  'instance of': 'Q5',
  'sport': 'Q1455'},
 'wesley pionteck souza': {'country of citizenship': 'Q155',
  'place of birth': 'Q1759384',
  'instance of': 'Q5',
  'position 